<a href="https://colab.research.google.com/github/MathotaGamma/AI/blob/main/AI4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install diffusers transformers accelerate

import os
import torch
from diffusers import StableDiffusionPipeline, StableDiffusionImg2ImgPipeline
from PIL import Image


model_list = ['stablediffusionapi/eleet-model','stablediffusionapi/brav6','Vsukiyaki/ShiratakiMix']
model_id = ''
while True:
  set = input('設定(未入力でOK)')
  if set == 'model':
    model_id = input('model_idを入力')
  elif set == 'model_list':
    for k in range(len(model_list)):
      print(str(1+k)+':'+model_list[k])
    model_id = model_list[int(input('何番を使用しますか？'))-1]

  elif set != '':
    print('設定が見つかりません。')
    continue
  else:
    break



if model_id == '':
  # Stable Diffusionのモデルをロード
  model_id = "stablediffusionapi/eleet-model"


# ユーザーからの入力を取得
prompt = input("プロンプトを入力してください (強さをつける場合は形式: 'prompt1:weight, prompt2:weight'): ")
negative_prompt = input("ネガティブプロンプトを入力してください: ")
num_images = int(input("生成する画像の枚数を入力してください: "))
num_inference_steps = int(input("推論ステップ数を入力してください: "))
mode = input("text to image または image to image を選んでください (t/i): ")

!mkdir output

# フォルダ名をプロンプトの前15文字にする
output_dir = 'output/'+prompt[:15].replace(" ", "_")  # スペースをアンダースコアに置き換え
os.makedirs(output_dir, exist_ok=True)

# 画像生成
if mode.lower() == 't':
    pipe = StableDiffusionPipeline.from_pretrained(
        model_id,
        torch_dtype=torch.float16,
        safety_checker=None  # Safe checkを無効に
    )
    pipe = pipe.to("cuda")  # GPUが必要です
    ever_num_images = 0
    while True:
        for i in range(num_images):
            image = pipe(
                prompt=prompt,
                negative_prompt=negative_prompt,
                num_inference_steps=num_inference_steps
            ).images[0]
            # 画像をファイルに保存 (数字のファイル名)
            image.save(os.path.join(output_dir, f"{i + 1}.png"))
        k_num_images = int(input('追加で生成する場合はその数を入れてください。(0で終了)'))
        if(k_num_images == 0):
            break
        else:
            num_inference_steps = int(input("推論ステップ数を入力してください: "))
            ever_num_images += num_images
            num_images = k_num_images
elif mode.lower() == 'i':
    image_path = input("元画像のpathを入力してください: ")
    strength = float(input("元画像の変更量(0~1)"))
    original_image = Image.open(image_path).convert("RGB")

    # Image-to-Imageパイプライン
    pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
        model_id,
        torch_dtype=torch.float16,
        safety_checker=None
    )
    pipe = pipe.to("cuda")
    ever_num_images = 0
    while True:
        for i in range(num_images):
            image = pipe(
                prompt=prompt,
                negative_prompt=negative_prompt,
                image=original_image,
                strength=strength,  # 元画像をどれだけ変化させるか (0～1)
                num_inference_steps=num_inference_steps
            ).images[0]
            # 画像をファイルに保存 (数字のファイル名)
            image.save(os.path.join(output_dir, f"{ever_num_images + i + 1}.png"))
        k_num_images = int(input('追加で生成する場合はその数を入れてください。(0で終了)'))
        if(k_num_images == 0):
            break
        else:
            strength = float(input('元画像の変更量(0~1)'))
            ever_num_images += num_images
            num_images = k_num_images


print(f"生成された画像は '{output_dir}' フォルダに保存されました。")


KeyboardInterrupt: Interrupted by user

In [ ]:
# AIを使って画像にキャプションを付ける
!pip install open_clip_torch
import open_clip  # pip install open_clip_torch
import torch
from PIL import Image

# GPUが利用できる場合はGPUを使う
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# AIモデルの読み込み
model, _, transform = open_clip.create_model_and_transforms(
    "coca_ViT-L-14",
    pretrained="mscoco_finetuned_laion2B-s13B-b90k",
    device=device,
)

# 画像の読み込み（必要に応じてファイル名を変更）
img = Image.open("IMG_3122.png").convert("RGB")

# 画像からキャプションを生成
im = transform(img).unsqueeze(0).to(device)
with torch.no_grad(), torch.cuda.amp.autocast():
    generated = model.generate(im, seq_len=20)

# キャプションを人間が読める文章に変換して表示
caption = (
    open_clip.decode(generated[0].detach())
    .split("<end_of_text>")[0]
    .replace("<start_of_text>", "")
)
print(caption)


<ipython-input-2-1d09104d5cc1>:22: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.no_grad(), torch.cuda.amp.autocast():


a black and white image of a japanese text . 


In [ ]:
!rm -r output

In [ ]:
!mkdir output